In [1]:
# Load API_key from .env file
from config_loader import ConfigLoader
config = ConfigLoader()

In [2]:
from typing import Annotated

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.checkpoint.memory import MemorySaver

def select_model():
    choice_map = {
        "1": ("Claude", ChatAnthropic(model="claude-3-5-sonnet-20240620")),
        "2": ("GPT-3", ChatOpenAI(model_name="gpt-3.5-turbo-1106"))
    }

    while (choice := input("Enter 1 for Claude or 2 for GPT-3: ").strip()) not in choice_map:
        print("Invalid choice. Please enter 1 or 2.")

    model_name, llm_instance = choice_map[choice]
    print(f"You selected {model_name}.")
    return llm_instance


class State(TypedDict):
    messages: Annotated[list, add_messages]    

graph_builder = StateGraph(State)

#Add New function; Tools
tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = select_model()
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
memory = MemorySaver()
#This is new
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"],
    # interrupt_after=["tools"]
)

Enter 1 for Claude or 2 for GPT-3:  1


You selected Claude.


In [3]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================

[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01V959jiH8mxZ97YVXEHyDVJ', 'input': {'query': 'LangGraph framework for language models'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01V959jiH8mxZ97YVXEHyDVJ)
 Call ID: toolu_01V959jiH8mxZ97YVXEHyDVJ
  Args:
    query: LangGraph framework for language models


In [4]:
snapshot = graph.get_state(config)
existing_message = snapshot.values["messages"][-1]
existing_message.pretty_print()

================================== Ai Message ==================================

[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01V959jiH8mxZ97YVXEHyDVJ', 'input': {'query': 'LangGraph framework for language models'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01V959jiH8mxZ97YVXEHyDVJ)
 Call ID: toolu_01V959jiH8mxZ97YVXEHyDVJ
  Args:
    query: LangGraph framework for language models


In [5]:
#Tool을 통해 검색하려고 하는데 대신 사용자가 메세지를 집어넣어서 해보자
from langchain_core.messages import AIMessage, ToolMessage

answer = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs. Above all, Yoon likes this LangGraph"
)
new_messages = [
    # The LLM API expects some ToolMessage to match its tool call. We'll satisfy that here.
    ToolMessage(content=answer, tool_call_id=existing_message.tool_calls[0]["id"]),
    # And then directly "put words in the LLM's mouth" by populating its response.
    AIMessage(content=answer),
]

new_messages[-1].pretty_print()
graph.update_state(
    # Which state to update
    config,
    # The updated values to provide. The messages in our `State` are "append-only", meaning this will be appended
    # to the existing state. We will review how to update existing messages in the next section!
    {"messages": new_messages},
)

print(graph.get_state(config).values["messages"])



================================== Ai Message ==================================

LangGraph is a library for building stateful, multi-actor applications with LLMs. Above all, Yoon likes this LangGraph


{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efa83b4-e41e-6506-8002-ff39cf843f6b'}}

In [6]:
print(graph.get_state(config).values["messages"])


[HumanMessage(content="I'm learning LangGraph. Could you do some research on it for me?", additional_kwargs={}, response_metadata={}, id='a41397bc-ac0a-495d-827b-2c78a7bfbd91'), AIMessage(content=[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01V959jiH8mxZ97YVXEHyDVJ', 'input': {'query': 'LangGraph framework for language models'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_0133GmJkvzgRfGMHYHaTCQE7', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 414, 'output_tokens': 115}}, id='run-2cf75201-6f9e-4a11-83a4-d02eae6be781-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph framework for language models'}, 'id': 'toolu_01V959jiH8mxZ97YVXEH